In [1]:
# Step 8: Baseline Centralized MLP Model Training

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


2025-06-12 17:55:10.982839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
### after cleanup
# Load the cleaned centralized dataset
df = pd.read_csv('../datasets/diabetes/cleaned_data.csv')

In [5]:
# Correct binary label: 0 = NO, 1 = <30 or >30
df['readmitted_binary'] = df['readmitted'].apply(lambda x: 0 if x == 0 else 1)

# Drop target and leaky column(s)
df = df.drop(columns=['readmitted', 'discharge_disposition_id'])

# Prepare features and target
X = df.drop(columns=['readmitted_binary'])
y = df['readmitted_binary']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Build MLP model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/home/tesla/anaconda3/envs/tff-env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
E0000 00:00:1749563578.251349   24585 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1749563578.252727   24585 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
# Train
history = model.fit(X_train_scaled, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    verbose=2,)

Epoch 1/20
2239/2239 - 7s - 3ms/step - accuracy: 0.8859 - loss: 0.3509 - val_accuracy: 0.8876 - val_loss: 0.3442
Epoch 2/20
2239/2239 - 6s - 3ms/step - accuracy: 0.8877 - loss: 0.3407 - val_accuracy: 0.8881 - val_loss: 0.3432
Epoch 3/20
2239/2239 - 6s - 3ms/step - accuracy: 0.8878 - loss: 0.3390 - val_accuracy: 0.8872 - val_loss: 0.3424
Epoch 4/20
2239/2239 - 5s - 2ms/step - accuracy: 0.8878 - loss: 0.3377 - val_accuracy: 0.8879 - val_loss: 0.3436
Epoch 5/20
2239/2239 - 5s - 2ms/step - accuracy: 0.8878 - loss: 0.3372 - val_accuracy: 0.8878 - val_loss: 0.3415
Epoch 6/20
2239/2239 - 5s - 2ms/step - accuracy: 0.8879 - loss: 0.3364 - val_accuracy: 0.8876 - val_loss: 0.3430
Epoch 7/20
2239/2239 - 6s - 3ms/step - accuracy: 0.8879 - loss: 0.3361 - val_accuracy: 0.8879 - val_loss: 0.3446
Epoch 8/20
2239/2239 - 6s - 2ms/step - accuracy: 0.8881 - loss: 0.3356 - val_accuracy: 0.8874 - val_loss: 0.3444
Epoch 9/20
2239/2239 - 6s - 3ms/step - accuracy: 0.8883 - loss: 0.3353 - val_accuracy: 0.8879 - 

In [8]:
# Evaluate
y_pred_probs = model.predict(X_test_scaled).flatten()
y_pred = (y_pred_probs > 0.7).astype(int) 

622/622 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  


In [14]:
# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)


Test Accuracy: 0.8817
Test Precision: 0.8925
Test Recall: 0.9854
Test F1 Score: 0.9366
Confusion Matrix:
[[  137  2097]
 [  258 17407]]
